In [2]:
import os
import json
import openai
from dotenv import load_dotenv
import time

In [5]:
load_dotenv()

True

In [6]:
openai.api_key = os.environ["OPENAI_API_KEY"]

In [29]:
with open("./data/2023_11_KICE.json", "rb") as f:
    data = json.load(f)

In [30]:
total_score = 0
for pa in data:
    for problem in pa["problems"]:
        total_score += problem["score"]
assert (total_score == 100)
print("test passed")

test passed


In [31]:
for pa in data:
    for problem in pa["problems"]:
        print(
            f'질문 : {problem["question"]}\n1번 : {problem["choices"][0]}\n2번 : {problem["choices"][1]}\n3번 : {problem["choices"][2]}\n4번 : {problem["choices"][3]}\n5번 : {problem["choices"][4]}\n정답 : {problem["answer"]}\n점수 : {problem["score"]}\n\n')

질문 : 윗글의 내용과 일치하지 않는 것은?
1번 : 같은 책을 읽은 독자라도 서로 다른 의미를 구성할 수 있다.
2번 : 다른 독자와의 소통은 독자가 인식의 폭을 확장하도록 돕는다
3번 : 독자는 직접 경험해 보지 못했던 다양한 삶을 책의 필자를 매개로 접할 수 있다.
4번 : 독자의 배경지식, 관점, 읽기 환경, 과제는 독자의 의미 구성에 영향을 주는 독자 요인이다.
5번 : 독자는 책을 읽을 때 자신이 속한 사회나 시대의 영향을 받으며 필자와 간접적으로 대화한다
정답 : 4
점수 : 2


질문 : 다음은 학생이 독서 후 작성한 글의 일부이다. [A]를 바탕으로 ⓐ～ⓔ를 이해한 내용으로 가장 적절한 것은?
1번 : ⓐ와 ⓑ에는 모두 ‘독자 자신에게서 답을 찾는 질문’이 나타난다.
2번 : ⓒ와 ⓓ에는 모두 ‘책에 명시된 내용’에서 질문의 답을 찾아내는 모습이 나타난다.
3번 : ⓐ에는 ‘책에서 답을 찾는 질문’이, ⓔ에는 그에 대한 답을 ‘독자의 경험’에서 찾아내는 모습이 나타난다.
4번 : ⓑ에는 ‘책에서 답을 찾는 질문’이, ⓒ에는 그에 대한 답을 ‘책의 내용들을 관계 지으며’ 찾아내는 모습이 나타난다.
5번 : ⓓ에는 ‘독자 자신에게서 답을 찾는 질문’이, ⓔ에는 그에 대한 답을 ‘독자의 경험’에서 찾아내는 모습이 나타난다.
정답 : 5
점수 : 3


질문 : 윗글을 읽고 ㉠에 대해 보인 반응으로 적절하지 않은 것은?
1번 : 스스로 독서 계획을 세우고 자신에게 필요한 책을 찾아 개인적으로 읽는 과정에서 경험할 수 있겠군.
2번 : 독서 모임에서 서로 다른 관점을 확인하고 자신의 관점을 조정하는 과정에서 경험할 수 있겠군.
3번 : 개인적으로 형성한 의미를, 독서 동아리를 통해 심화하는 과정에서 경험할 수 있겠군.
4번 : 자신의 독서 경험을 담은 콘텐츠를 생산하고 공유하는 과정에서 경험할 수 있겠군.
5번 : 오프라인뿐 아니라 온라인 공간에서 해석을 나누는 과정에서도 경험할 수 있겠군.
정답 : 1
점수 : 2


질문 : (가)와 (나)에 대한

# Test

In [20]:
def get_answer_basic(paragraph, question, choices, question_plus="", get_prompt=False, no_paragraph=False):
    system_prompt = """
        국어 시험 문제를 푸는 천재 수험생으로써 다음 문제의 답을 구하세요.
    """
    if not no_paragraph:
        user_prompt = f"""
            지문 :
            {paragraph}
        """
    else:
        user_prompt = ""
    if question_plus:
        user_prompt += f"""
            <보기> :
            {question_plus}
        """
    user_prompt += f"""
        질문 :
        {question}

        선택지 :
        1번 - {choices[0]}
        2번 - {choices[1]}
        3번 - {choices[2]}
        4번 - {choices[3]}
        5번 - {choices[4]}

        1번, 2번, 3번, 4번, 5번 중 정답을 말하세요. 풀이는 말하지 마세요.
    """
    if get_prompt:
        return system_prompt +"\n\n" +user_prompt

    completion = openai.ChatCompletion.create(model="gpt-4", messages=[{
        "role": "system", "content": system_prompt
    }, {
        "role": "user", "content": user_prompt
    }], temperature=0.0)
    return completion.choices[0].message.content

In [18]:
def get_answer_true(paragraph, question, choices, question_plus="", get_prompt=False, no_paragraph=False):
    system_prompt = """
        국어 시험 문제를 푸는 천재 선생님으로써 각 문제의 선택지에 대한 해설을 제작하여야 합니다.
    """
    if not no_paragraph:
        user_prompt = f"""
            지문 :
            {paragraph}
        """
    else:
        user_prompt = ""
    if question_plus:
        user_prompt += f"""
            문제의 각 선택지들을 해결하기 위한 배경 지식을 설명해 주고 있는 것이 <보기>로써, 각 선택지들을 지문과 연결시키고, <보기>의 지식을 활용하면 각 선택지의 참과 거짓을 판단할 수 있습니다.
            <보기> :
            {question_plus}
        """
    user_prompt += f"""
        질문 :
        {question}

        선택지 :
        1번 - {choices[0]}
        2번 - {choices[1]}
        3번 - {choices[2]}
        4번 - {choices[3]}
        5번 - {choices[4]}

        이제, 1번부터 5번까지 모든 선택지가 참이라고 가정하세요. 모든 선택지가 참인 것에는 지문 혹은 보기에서 찾은 적절한 근거와 이유가 존재합니다.
        해당 논리와 근거들을 서술하세요.
        아래 양식을 따라야 합니다.

        X번
        선택지가 참인 것을 증명하는 지문 혹은 보기의 근거 : (지문 혹은 보기에서 찾은 문장)
        참인 이유 : (참인 이유)

        ...(1번부터 5번까지 반복)
    """
    if get_prompt:
        return system_prompt +"\n\n" +user_prompt

    completion = openai.ChatCompletion.create(model="gpt-4", messages=[{
        "role": "system", "content": system_prompt
    }, {
        "role": "user", "content": user_prompt
    }], temperature=0.0)
    return completion.choices[0].message.content

In [93]:
def get_answer_false(paragraph, question, choices, question_plus="", get_prompt=False, no_paragraph=False):
    system_prompt = """
        국어 시험 문제를 푸는 천재 선생님으로써 각 문제의 선택지에 대한 해설을 제작하여야 합니다.
    """
    if not no_paragraph:
        user_prompt = f"""
            지문 :
            {paragraph}
        """
    else:
        user_prompt = ""
    if question_plus:
        user_prompt += f"""
            문제의 각 선택지들을 해결하기 위한 배경 지식을 설명해 주고 있는 것이 <보기>로써, 각 선택지들을 지문과 연결시키고, <보기>의 지식을 활용하면 각 선택지의 참과 거짓을 판단할 수 있습니다.
            <보기> :
            {question_plus}
        """
    user_prompt += f"""
        질문 :
        {question}

        선택지 :
        1번 - {choices[0]}
        2번 - {choices[1]}
        3번 - {choices[2]}
        4번 - {choices[3]}
        5번 - {choices[4]}

        이제, 1번부터 5번까지 모든 선택지가 거짓이라고 가정하세요. 모든 선택지가 거짓인 것에는 지문 혹은 보기에서 찾은 적절한 근거와 이유가 존재합니다.
        해당 논리와 근거들을 서술하세요.
        아래 양식을 따라야 합니다.

        X번
        선택지가 거짓인 것을 증명하는 지문 혹은 보기의 근거 : (지문 혹은 보기에서 찾은 문장)
        거짓인 이유 : (거짓인 이유)

        ...(1번부터 5번까지 반복)...
    """
    if get_prompt:
        return system_prompt +"\n\n" +user_prompt

    completion = openai.ChatCompletion.create(model="gpt-4", messages=[{
        "role": "system", "content": system_prompt
    }, {
        "role": "user", "content": user_prompt
    }], temperature=0.0)
    return completion.choices[0].message.content

In [91]:
def get_answer_false_plus(paragraph, question, choices, question_plus="", get_prompt=False, no_paragraph=False):
    system_prompt = """
        국어 시험 문제를 푸는 천재 선생님으로써 각 문제의 선택지에 대한 해설을 제작하여야 합니다.
    """
    if not no_paragraph:
        user_prompt = f"""
            지문 :
            {paragraph}
        """
    else:
        user_prompt = ""
    if question_plus:
        user_prompt += f"""
            문제의 각 선택지들을 해결하기 위한 배경 지식을 설명해 주고 있는 것이 <보기>로써, 각 선택지들을 지문과 연결시키고, <보기>의 지식을 활용하면 각 선택지의 참과 거짓을 판단할 수 있습니다.
            <보기> :
            {question_plus}
        """
    user_prompt += f"""
        질문 :
        {question}

        선택지 :
        1번 - {choices[0]}
        2번 - {choices[1]}
        3번 - {choices[2]}
        4번 - {choices[3]}
        5번 - {choices[4]}

        이제, 1번부터 5번까지 모든 선택지는 거짓입니다. 모든 선택지가 거짓인 것에는 지문 혹은 보기에서 찾은 적절한 근거와 이유가 있습니다.
        거짓인 이유는 아래와 같은 유형이 있습니다.
        - 단순 사실 관계가 맞지 않을 때
        - 지문 및 보기의 내용과 선택지의 주장이 논리적으로 맞지 않을 때
        - 지문의 전체적인 내용과 선택지의 주장이 맞지 않을 때
        - 지문을 기반으로 논리적으로 판단하였을 때 선택지의 주장이 거짓일 때
        선택지가 거짓임을 증명하세요.
        모든 선택지의 내용은 지문이나 보기에서 무조건 언급되어 있거나 그것을 바탕으로 추론할 수 있습니다.
        거짓인 이유를 논리적으로 서술해야 합니다. 단순히 유형을 서술하는 것은 인정되지 않습니다.
        아래 양식을 따라야 합니다.

        X번
        선택지가 거짓인 것을 증명하는 지문 혹은 보기의 근거 : (지문 혹은 보기에서 찾은 문장) '없는 경우는 없습니다'
        거짓인 이유 : (거짓인 이유)

        ...(1번부터 5번까지 반복)...
    """
    if get_prompt:
        return system_prompt +"\n\n" +user_prompt

    completion = openai.ChatCompletion.create(model="gpt-4", messages=[{
        "role": "system", "content": system_prompt
    }, {
        "role": "user", "content": user_prompt
    }], temperature=0.0)
    return completion.choices[0].message.content

In [23]:
def get_answer_judge(paragraph, question, choices, question_plus="", get_prompt=False, no_paragraph=False):
    system_prompt = """
        국어 시험 문제를 푸는 천재 선생님으로써 각 문제의 선택지에 대해 참이라고 주장하는 근거와 거짓이라고 주장하는 근거를 보고, 각 선택지의 실제 참과 거짓을 분별하여 최종적으로 답을 찾아내야 합니다.
        답을 찾기 위해서 아래 지침을 따르세요.

        <지침>
        1번부터 5번까지 각 선택지에 대한 참과 거짓을 판단하세요.
        문제의 질문이 "가장 적절한 것은?"으로 끝나면 유일한 참인 선택지를 선택하고, "적절하지 않은 것은?"으로 끝나면 유일한 거짓인 선택지가 답입니다.

        지문과 질문, 선택지 이후에는 각 선택지가 참이라고 생각하는 이유와 근거, 각 선택지가 거짓이라고 생각하는 이유와 근거가 주어집니다.
        해당 이유와 근거들을 활용하여 최종적인 각 선택지의 참과 거짓을 분류하세요.

        실제로는 참이지 않거나 거짓이지 않은 이유와 근거들은 주로 다음과 같은 특징을 가지고 있습니다.
        - 이유가 근거를 바탕으로 하지 않은 경우
        - 이유가 단순히 선택지를 반복할 경우
        - 이유가 지문을 바탕으로 할 때 논리적으로 타당하지 않을 경우
        - 선택지와 관련 없는 근거일 경우
        - 실제로 지문 혹은 보기에 없는 근거일 경우

        해당 특징이 나타나는지 주의하며, 각 선택지의 참과 거짓을 판별하세요.
    """
    if not no_paragraph:
        user_prompt = f"""
            지문 :
            {paragraph}
        """
    else:
        user_prompt = ""
    if question_plus:
        user_prompt += f"""
            문제의 각 선택지들을 해결하기 위한 배경 지식을 설명해 주고 있는 것이 <보기>로써, 각 선택지들을 지문과 연결시키고, <보기>의 지식을 활용하면 각 선택지의 참과 거짓을 판단할 수 있습니다.
            <보기> :
            {question_plus}
        """
    user_prompt += f"""
        질문 :
        {question}

        선택지 :
        1번 - {choices[0]}
        2번 - {choices[1]}
        3번 - {choices[2]}
        4번 - {choices[3]}
        5번 - {choices[4]}

        각 선택지가 참인 이유 :

        각 선택지가 거짓인 이유 :

        위 이유들을 고려하여, 문제에 대한 정답을 말하세요. 그렇게 생각한 이유도 서술하세요.
    """
    if get_prompt:
        return system_prompt +"\n\n" +user_prompt

    completion = openai.ChatCompletion.create(model="gpt-4", messages=[{
        "role": "system", "content": system_prompt
    }, {
        "role": "user", "content": user_prompt
    }], temperature=0.0)
    return completion.choices[0].message.content

In [97]:
def get_answer_bwook_king(paragraph, question, choices, question_plus="", get_prompt=False, no_paragraph=False):
    system_prompt = """
        국어 시험 문제를 푸는 대한민국의 고3 수험생으로서 위의 요약을 바탕으로 다음 문제의 답을 구하세요.

         문제를 풀이할 때, 반드시 지문을 참고하세요.
         문제는 무조건 1개의 정답만 있습니다.
         문제를 풀이할 때 모든 선택지들을 검토하세요.
         모든 선택지마다 근거를 지문에서 찾아 설명하세요.

         다음의 형식을 따라 답변하세요.
         최종 정답: (최종 정답)
         1번: (선택지 1번에 대한 답변) + "(지문 속 근거가 된 문장)"
         2번: (선택지 2번에 대한 답변) + "(지문 속 근거가 된 문장)"
         3번: (선택지 3번에 대한 답변) + "(지문 속 근거가 된 문장)"
         4번: (선택지 4번에 대한 답변) + "(지문 속 근거가 된 문장)"
         5번: (선택지 5번에 대한 답변) + "(지문 속 근거가 된 문장)"

    """
    user_prompt = f"""
        지문 :
        {paragraph}
    """
    if question_plus:
        user_prompt += f"""
            이 문제는 아래와 같이 <보기>가 주어져 있습니다. 문제의 각 선택지들을 해결하기 위한 배경 지식을 설명해 주고 있는 것이 <보기>로써, 각 선택지들을 지문과 연결시키고, <보기>의 지식을 활용하면 각 선택지의 참과 거짓을 판단할 수 있습니다.
            문제를 해결할 때, 반드시 <보기>의 내용을 이용해서 문제를 해결해야 합니다.
            <보기> :
            {question_plus}
        """
    user_prompt += f"""
        질문 :
        {question}

        선택지 :
        1번 - {choices[0]}
        2번 - {choices[1]}
        3번 - {choices[2]}
        4번 - {choices[3]}
        5번 - {choices[4]}

    """
    if get_prompt:
        return system_prompt +"\n\n" +user_prompt

    completion = openai.ChatCompletion.create(model="gpt-4", messages=[{
        "role": "system", "content": system_prompt
    }, {
        "role": "user", "content": user_prompt
    }], temperature=0.0)
    return completion.choices[0].message.content

In [98]:
def get_answer_one_problem(paragraph_num: int, problem_num: int, get_prompt=False, answer_func:callable = get_answer_basic):
    problem = data[paragraph_num]["problems"][problem_num]
    no_paragraph = False
    if "no_paragraph" in list(problem.keys()):
        no_paragraph = True
    if "question_plus" in list(problem.keys()):
        question_plus_text = problem["question_plus"]
    else:
        question_plus_text = ""
    return answer_func(paragraph=data[paragraph_num]["paragraph"],
                      question=problem["question"],
                      choices=problem["choices"],
                      question_plus=question_plus_text,
                      get_prompt=get_prompt,
                      no_paragraph=no_paragraph)

In [99]:
import clipboard
clipboard.copy(get_answer_one_problem(4,3, get_prompt=True, answer_func=get_answer_bwook_king))

In [100]:
print(get_answer_one_problem(4,3, get_prompt=False, answer_func=get_answer_bwook_king))

최종 정답: 5번
1번: 옥영의 꿈에 나타난 ‘만복사의 부처’는, 옥영이 겪고 있는 현실적인 문제를 해결하는 데 도움을 주는 신이한 존재로서 역할을 한다고 볼 수 있겠군. - "이날 밤 장육금불이 옥영의 꿈에 나타나 말했다.“나는 만복사의 부처로다. 너희 정성이 가상해 기이한 사내아이를 점지해 주니, 태어나면 반드시 특이한 징표가 있을 것이다.”
2번: 몽석의 몸에 나타난 ‘붉은 점’은, ‘사내아이’의 출생과 관련한 예언이 실제로 이루어졌음을 확인할 수 있는 특이한 증거로 활용된다고 볼 수 있겠군. - "옥영은 ㉢(그달)에 바로 잉태해 열 달 뒤 과연 아들을 낳았는데, 등에 어린아이 손바닥만 한 붉은 점이 있었다."
3번: 최척이 ‘일본인 배에 이르러 조선말로 물’어보는 것과 ‘고국 사람을 만나’려 하는 것은, 서사 전개 과정에서 공간적 배경을 조선뿐 아니라 다른 나라로도 확장한 것과 관련이 있겠군. - "최척은 동방이 밝아 오자, 강둑을 내려가 일본인 배에 이르러조선말로 물었다."
4번: 옥영이 들은 ‘피리 소리’는, 옥영이 최척을 떠올리게 하여 이별의 상황을 해결하는 계기가 되는 소재로 작용하고 있다고 볼 수 있겠군. - "옥영도 생각하기를 어젯밤 들은 피리 소리가 조선의 곡조인데다, 평소 익히 들었던 것과 너무나 흡사했다."
5번: 최척과 옥영이 ‘소리를 지르며 끌어안’는 것은 문제의 해결에 따른 기쁨과, ‘눈물이 다하자 피가 흘러내’리는 것은 또 다른 문제 확인에 따른 인물의 불안감과 관련이 있겠군. - "둘은 서로 마주하고 놀라 소리를 지르며 끌어안고 백사장을 뒹굴었다. 목이 메고 기가 막혀 마음을 안정할 수 없었으며, 말도 할 수 없었다. 눈에서는 눈물이 다하자 피가 흘러내려 서로를 볼 수도 없을 지경이었다." 이 부분은 또 다른 문제가 확인되는 것이 아니라, 기존의 문제가 해결되는 장면이므로 선택지 5번은 적절하지 않다.


In [101]:
from tqdm.notebook import tqdm
import time
answers_list = list()
for paragraph_index, paragraph in enumerate(data):
    for problem_index, problem in tqdm(enumerate(paragraph["problems"])):
        answers_list.append(get_answer_one_problem(paragraph_index, problem_index, answer_func=get_answer_bwook_king))
        time.sleep(20)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [102]:
answers_list

['최종 정답: 4번\n1번: "같은 책을 읽은 독자라도 독자의 배경지식이나 관점 등의 독자 요인, 읽기 환경이나 과제 등의 상황 요인이 다르므로, 필자가 보여 주는 세계를 그대로 수용하지 않고 저마다 소통 과정에서 다른 의미를 구성할 수 있다."라는 문장에서 확인할 수 있습니다.\n2번: "비슷한 해석에 서로 공감하며 기존 인식을 강화하거나 관점의 차이를 확인하고 기존 인식을 조정하는 과정에서, 독자는자신의 인식을 심화 확장할 수 있다."라는 문장에서 확인할 수 있습니다.\n3번: "독자는 자신이 속한사회나 시대의 영향 아래 필자가 속해 있거나 드러내고자 하는 사회나 시대를 경험한다. 직접 경험하지 못했던 다양한 삶을 필자를 매개로 만나고 이해하면서 독자는 더 넓은 시야로 세계를바라볼 수 있다."라는 문장에서 확인할 수 있습니다.\n4번: 지문에서 "독자의 배경지식, 관점, 읽기 환경, 과제는 독자의 의미 구성에 영향을 주는 독자 요인이다."라는 내용은 나오지 않았습니다.\n5번: "독자는 자신이 속한사회나 시대의 영향 아래 필자가 속해 있거나 드러내고자 하는 사회나 시대를 경험한다."라는 문장에서 확인할 수 있습니다.',
 "최종 정답: 5번\n1번: ⓐ와 ⓑ에는 모두 ‘독자 자신에게서 답을 찾는 질문’이 나타난다. - 이 선택지는 부정확하다. ⓐ는 베토벤의 교향곡 <합창>이 위대한 작품인 이유에 대한 질문으로, 이는 '책에서 답을 찾는 질문'에 해당한다. ⓑ는 책에서 얻은 정보를 바탕으로 '낭만주의 음악의 특징'에 대한 이해를 나타내는 부분으로, 이 역시 '책에서 답을 찾는 질문'에 해당한다. 따라서 이 선택지는 부정확하다.\n2번: ⓒ와 ⓓ에는 모두 ‘책에 명시된 내용’에서 질문의 답을 찾아내는 모습이 나타난다. - 이 선택지는 부정확하다. ⓒ는 책의 내용을 종합하여 베토벤의 교향곡 구성의 새로움에 대한 이해를 나타내는 부분으로, 이는 '책에서 답을 찾는 질문'에 해당한다. 그러나 ⓓ는 학생이 자신에게 진정으로 좋아하는 일이 무엇인지 묻는 부분으로, 이

In [16]:
get_answer_one_problem(0,0)

'1번'

In [9]:
#21번 문제 - 정답 5번
get_answer_one_problem(4,3, get_prompt=True).replace("\n", "")

'        국어 시험 문제를 푸는 천재 수험생으로써 다음 문제의 답을 구하세요.        답을 구하는 방법은 다음과 같습니다. 해당 문제에서 5개의 선택지가 주어집니다.        각각의 선택지는 각각 참 (예)이거나 거짓 (아니오)입니다. 애매하거나 답을 구할 수 없는 선택지는 존재하지 않습니다.        1번부터 5번까지 각 선택지에 대한 참과 거짓을 판단하세요.        문제의 질문에 따라 유일한 참인 선택지를 선택하거나, 유일한 거짓인 선택지를 선택해야 합니다.        문제의 질문이 "가장 적절한 것은?"으로 끝나면 유일한 참인 선택지를 선택하세요.        반대로, 문제의 질문이 "적절하지 않은 것은?"으로 끝나면 유일한 거짓인 선택지를 선택하세요.        문제를 풀기 위하여, 먼저 주어진 지문의 정보를 참고하여 각 선택지에 대해서 참과 거짓을 구별하세요. 구별할 때에는 지문에 제시되어 있는 \'단서\'와 추론에 필요한 핵심적인 정보를 연결지어 간단히 설명하세요. 다만, 단순하게 지문의 내용의 사실 관계 만으로 1차원적인 정답을 구하지 않고, 실제 국어 문제를 풀이하는 선생님과 같은 관점에서 적절한 정답을 찾아가야 합니다.        이후, 최종적으로 유일한 참 혹은 유일한 거짓인 선택지를 고릅니다. 마지막으로, 적절한 답안은 "1번"과 같이 선택지의 번호만을 적으면 됩니다.        만약 유일한 참 혹은 유일한 거짓이 없다면, 적합한 사고 과정을 거치지 않거나 충분하게 지문을 참조하지 않은 것입니다. 하나의 선택지에 참고해야할 부분이 지문에서 여러 개가 있을 수 있으며, (가) 및 (나) 등을 비롯한 여러 글에서 참고해야할 부분이 있을 수 있습니다. 이러한 부분들을 반드시 모두 참고하여 문제를 풀이해야 합니다.        또한, 문제에서 [A]나 [B]를 언급한 경우, 이는 지문 안의 여러 문장들을 표시한 것입니다. 지문에서 [A]부분을 찾고, 그 이후에 소괄호 () 안에 들어있는 내용들을 참고하세요.     

In [31]:
# 29번 문제 - 정답 1번
get_answer_one_problem(6,2, get_prompt=True).replace("\n", "")

'        국어 시험 문제를 푸는 똑똑한 학생으로써 다음 문제의 답을 구하세요.        답을 구하는 방법은 다음과 같습니다. 해당 문제에서 5개의 선택지가 주어집니다.        각각의 선택지는 각각 참 (예)이거나 거짓 (아니오)입니다. 애매하거나 답을 구할 수 없는 선택지는 존재하지 않습니다.        1번부터 5번까지 각 선택지에 대한 참과 거짓을 판단하세요.        문제의 질문에 따라 유일한 참인 선택지를 선택하거나, 유일한 거짓인 선택지를 선택해야 합니다.        문제의 질문이 "가장 적절한 것은?"으로 끝나면 유일한 참인 선택지를 선택하세요.        반대로, 문제의 질문이 "적절하지 않은 것은?"으로 끝나면 유일한 거짓인 선택지를 선택하세요.        문제를 풀기 위하여, 먼저 주어진 지문의 정보를 참고하여 각 선택지에 대해서 참과 거짓을 구별하세요. 구별할 때에는 지문에 제시되어 있는 \'증거\'와 그 추론 과정을 간단히 설명하세요. 다만, 단순하게 지문의 내용의 사실 관계 만으로 1차원적인 정답을 구하지 않고, 실제 국어 문제를 풀이하는 선생님과 같은 관점에서 적절한 정답을 찾아가야 합니다.        이후, 최종적으로 유일한 참 혹은 유일한 거짓인 선택지를 고릅니다. 마지막으로, 적절한 답안은 "1번"과 같이 선택지의 번호만을 적으면 됩니다.        만약 유일한 참 혹은 유일한 거짓이 없다면, 적합한 사고 과정을 거치지 않거나 충분하게 지문을 참조하지 않은 것입니다. 하나의 선택지에 참고해야할 부분이 지문에서 여러 개가 있을 수 있으며, (가) 및 (나) 등을 비롯한 여러 글에서 참고해야할 부분이 있을 수 있습니다. 이러한 부분들을 반드시 모두 참고하여 문제를 풀이해야 합니다.        또한, 문제에서 [A]나 [B]를 언급한 경우, 이는 지문 안의 여러 문장들을 표시한 것입니다. 지문에서 [A]부분을 찾고, 그 이후에 소괄호 () 안에 들어있는 내용들을 참고하세요.        ㉠~㉤ 등의 표시에

In [34]:
# 31번 문제 - 정답 1번
get_answer_one_problem(7,0, get_prompt=True).replace("\n", "")

'        국어 시험 문제를 풀이하는 똑똑한 한국어 선생님으로써 다음 문제의 답을 구하세요. 똑똑한 한국어 선생님은 기존의 한국어 지식, 문법, 어휘, 글의 구조, 풀이 방법 등을 이용하여 답을 구할 수 있습니다.        답을 구하는 방법은 다음과 같습니다. 해당 문제에서 5개의 선택지가 주어집니다.        각각의 선택지는 각각 참 (예)이거나 거짓 (아니오)입니다. 애매하거나 답을 구할 수 없는 선택지는 존재하지 않습니다.        1번부터 5번까지 각 선택지에 대한 참과 거짓을 판단하세요.        문제의 질문에 따라 유일한 참인 선택지를 선택하거나, 유일한 거짓인 선택지를 선택해야 합니다.        문제의 질문이 "가장 적절한 것은?"으로 끝나면 유일한 참인 선택지를 선택하세요.        반대로, 문제의 질문이 "적절하지 않은 것은?"으로 끝나면 유일한 거짓인 선택지를 선택하세요.        문제를 풀기 위하여, 먼저 주어진 지문의 정보를 참고하여 각 선택지에 대해서 참과 거짓을 구별하세요. 구별할 때에는 지문에 제시되어 있는 \'증거\'와 그 추론 과정을 간단히 설명하세요. 다만, 단순하게 지문의 내용의 사실 관계 만으로 1차원적인 정답을 구하지 않고, 실제 국어 문제를 풀이하는 선생님과 같은 관점에서 적절한 정답을 찾아가야 합니다.        이후, 최종적으로 유일한 참 혹은 유일한 거짓인 선택지를 고릅니다. 마지막으로, 적절한 답안은 "1번"과 같이 선택지의 번호만을 적으면 됩니다.        만약 유일한 참 혹은 유일한 거짓이 없다면, 적합한 사고 과정을 거치지 않거나 충분하게 지문을 참조하지 않은 것입니다. 하나의 선택지에 참고해야할 부분이 지문에서 여러 개가 있을 수 있으며, (가) 및 (나) 등을 비롯한 여러 글에서 참고해야할 부분이 있을 수 있습니다. 이러한 부분들을 반드시 모두 참고하여 문제를 풀이해야 합니다.        또한, 문제에서 [A]나 [B]를 언급한 경우, 이는 지문 안의 여러 문장들을 표

In [35]:
# 40번 문제 - 정답 3번
get_answer_one_problem(9,2, get_prompt=True).replace("\n", "")

'        국어 시험 문제를 풀이하는 똑똑한 한국어 선생님으로써 다음 문제의 답을 구하세요. 똑똑한 한국어 선생님은 기존의 한국어 지식, 문법, 어휘, 글의 구조, 풀이 방법 등을 이용하여 답을 구할 수 있습니다.        답을 구하는 방법은 다음과 같습니다. 해당 문제에서 5개의 선택지가 주어집니다.        각각의 선택지는 각각 참 (예)이거나 거짓 (아니오)입니다. 애매하거나 답을 구할 수 없는 선택지는 존재하지 않습니다.        1번부터 5번까지 각 선택지에 대한 참과 거짓을 판단하세요.        문제의 질문에 따라 유일한 참인 선택지를 선택하거나, 유일한 거짓인 선택지를 선택해야 합니다.        문제의 질문이 "가장 적절한 것은?"으로 끝나면 유일한 참인 선택지를 선택하세요.        반대로, 문제의 질문이 "적절하지 않은 것은?"으로 끝나면 유일한 거짓인 선택지를 선택하세요.        문제를 풀기 위하여, 먼저 주어진 지문의 정보를 참고하여 각 선택지에 대해서 참과 거짓을 구별하세요. 구별할 때에는 지문에 제시되어 있는 \'증거\'와 그 추론 과정을 간단히 설명하세요. 다만, 단순하게 지문의 내용의 사실 관계 만으로 1차원적인 정답을 구하지 않고, 실제 국어 문제를 풀이하는 선생님과 같은 관점에서 적절한 정답을 찾아가야 합니다.        이후, 최종적으로 유일한 참 혹은 유일한 거짓인 선택지를 고릅니다. 마지막으로, 적절한 답안은 "1번"과 같이 선택지의 번호만을 적으면 됩니다.        만약 유일한 참 혹은 유일한 거짓이 없다면, 적합한 사고 과정을 거치지 않거나 충분하게 지문을 참조하지 않은 것입니다. 하나의 선택지에 참고해야할 부분이 지문에서 여러 개가 있을 수 있으며, (가) 및 (나) 등을 비롯한 여러 글에서 참고해야할 부분이 있을 수 있습니다. 이러한 부분들을 반드시 모두 참고하여 문제를 풀이해야 합니다.        또한, 문제에서 [A]나 [B]를 언급한 경우, 이는 지문 안의 여러 문장들을 표

In [16]:
# 41번 문제 - 정답 2번
get_answer_one_problem(9,3, get_prompt=False).replace("\n", "")

'1번 선택지는 (나) 지문에서 "학생 2 : 근데 발표회까지 얼마 안 남아서 국가 보호종을 비롯해 주목할 만한 몇몇 식물만 표시해야 할 듯해. 그리고 식물 이름은 ○○ 고등학교처럼 누리집을 이용해 편리하게 찾자."라는 부분에서 확인할 수 있습니다. 따라서 1번은 참입니다.2번 선택지는 (나) 지문에서 "학생 2 : 근데 발표회까지 얼마 안 남아서 국가 보호종을 비롯해 주목할 만한 몇몇 식물만 표시해야 할 듯해."라는 부분에서 확인할 수 있습니다. 따라서 2번은 참입니다.3번 선택지는 (나) 지문에서 "학생 2 : 생각해 봤는데, 우리 셋이서 ○○ 고등학교가 한 것처럼 넓은 공간을 조사하긴 힘들 듯하니 학교에서 걸어갈 만한 거리만 지도의 범위로 삼는 게 좋지 않을까?"라는 부분에서 확인할 수 있습니다. 따라서 3번은 참입니다.4번 선택지는 (나) 지문에서 "학생 2 : 여러 종류의 식물이 있는 곳도 좋지만, 나는 우리 학교 학생들이 볼 지도이니 학생들에게 친숙한 장소가 더 좋을 듯해."라는 부분에서 확인할 수 있습니다. 따라서 4번은 참입니다.5번 선택지는 (나) 지문에서 "학생 2 : 그래. 그런데 ○○ 고등학교가 이어 붙이는 방식으로 지도를 만든 건 참신하긴 한데 통일감이 없어 부자연스러울 듯해. 우리는 조사한 내용을 모아 함께 지도를 그리자."라는 부분에서 확인할 수 있습니다. 따라서 5번은 참입니다.따라서, 모든 선택지가 참인 것으로 판단되므로, 문제에서 요구하는 \'적절하지 않은 것\'을 찾지 못했습니다. 이 경우, 지문을 다시 검토하거나, 다른 관점에서 선택지를 검토해야 합니다.'

In [15]:
get_answer_one_problem(0,1, get_prompt=False).replace("\n", "")

'1번 선택지는 ⓐ와 ⓑ에 모두 ‘독자 자신에게서 답을 찾는 질문’이 나타난다고 했는데, ⓐ는 \'음악 시간에 들었던 베토벤의 교향곡 <합창>이 위대한 작품인 이유는 무엇일까?\'라는 질문으로, 이는 책에서 답을 찾는 질문이다. ⓑ는 \'이 곡이 낭만주의 음악의 특징을 보여 준다고 했다.\'라는 책의 내용을 인용한 부분으로, 이 역시 책에서 답을 찾는 질문이다. 따라서 1번은 거짓이다.2번 선택지는 ⓒ와 ⓓ에 모두 ‘책에 명시된 내용’에서 질문의 답을 찾아내는 모습이 나타난다고 했는데, ⓒ는 \'절망적 상황에서도 열정적으로 자신이 좋아하는 일을 했기에 교향곡 구성의 새로움을 보여 준 명작이 탄생했음을 알게 됐다.\'라는 책의 내용을 종합하여 질문의 답을 찾아낸 부분이다. 하지만 ⓓ는 \'내가 진정으로 좋아하는 일이 무엇인지 나에게 묻게 되었다.\'라는 부분으로, 이는 독자 자신에게서 답을 찾는 질문이다. 따라서 2번은 거짓이다.3번 선택지는 ⓐ에는 ‘책에서 답을 찾는 질문’이, ⓔ에는 그에 대한 답을 ‘독자의 경험’에서 찾아내는 모습이 나타난다고 했는데, ⓐ는 \'음악 시간에 들었던 베토벤의 교향곡 <합창>이 위대한 작품인 이유는 무엇일까?\'라는 질문으로, 이는 책에서 답을 찾는 질문이다. ⓔ는 \'글 쓰는 일에서 가장 큰 행복을 느꼈던 나를 발견\'할 수 있었다는 부분으로, 이는 독자의 경험에서 답을 찾아낸 부분이다. 따라서 3번은 참이다.4번 선택지는 ⓑ에는 ‘책에서 답을 찾는 질문’이, ⓒ에는 그에 대한 답을 ‘책의 내용들을 관계 지으며’ 찾아내는 모습이 나타난다고 했는데, ⓑ는 \'이 곡이 낭만주의 음악의 특징을 보여 준다고 했다.\'라는 책의 내용을 인용한 부분으로, 이는 책에서 답을 찾는 질문이다. ⓒ는 \'절망적 상황에서도 열정적으로 자신이 좋아하는 일을 했기에 교향곡 구성의 새로움을 보여 준 명작이 탄생했음을 알게 됐다.\'라는 책의 내용을 종합하여 질문의 답을 찾아낸 부분이다. 따라서 4번은 참이다.5번 선택지는 ⓓ에는 ‘독자 자신에게서 답을

In [103]:
solutions = list()
for pa in data:
    for problem in pa["problems"]:
        solutions.append(problem["answer"])

In [104]:
scores = list()
for pa in data:
    for problem in pa["problems"]:
        scores.append(problem["score"])

In [106]:
f = open("./result/2023_11_KICE-wook_prompt_0714.txt", 'w', encoding='UTF-8')
for i, item in enumerate(answers_list):
    txt = f'{i+1}번 문제 : {item}\n정답 : {solutions[i]}\n배점 : {scores[i]}\n----------------------------\n'
    print(txt)
    f.write(txt)
f.close()

1번 문제 : 최종 정답: 4번
1번: "같은 책을 읽은 독자라도 독자의 배경지식이나 관점 등의 독자 요인, 읽기 환경이나 과제 등의 상황 요인이 다르므로, 필자가 보여 주는 세계를 그대로 수용하지 않고 저마다 소통 과정에서 다른 의미를 구성할 수 있다."라는 문장에서 확인할 수 있습니다.
2번: "비슷한 해석에 서로 공감하며 기존 인식을 강화하거나 관점의 차이를 확인하고 기존 인식을 조정하는 과정에서, 독자는자신의 인식을 심화 확장할 수 있다."라는 문장에서 확인할 수 있습니다.
3번: "독자는 자신이 속한사회나 시대의 영향 아래 필자가 속해 있거나 드러내고자 하는 사회나 시대를 경험한다. 직접 경험하지 못했던 다양한 삶을 필자를 매개로 만나고 이해하면서 독자는 더 넓은 시야로 세계를바라볼 수 있다."라는 문장에서 확인할 수 있습니다.
4번: 지문에서 "독자의 배경지식, 관점, 읽기 환경, 과제는 독자의 의미 구성에 영향을 주는 독자 요인이다."라는 내용은 나오지 않았습니다.
5번: "독자는 자신이 속한사회나 시대의 영향 아래 필자가 속해 있거나 드러내고자 하는 사회나 시대를 경험한다."라는 문장에서 확인할 수 있습니다.
정답 : 4
배점 : 2
----------------------------

2번 문제 : 최종 정답: 5번
1번: ⓐ와 ⓑ에는 모두 ‘독자 자신에게서 답을 찾는 질문’이 나타난다. - 이 선택지는 부정확하다. ⓐ는 베토벤의 교향곡 <합창>이 위대한 작품인 이유에 대한 질문으로, 이는 '책에서 답을 찾는 질문'에 해당한다. ⓑ는 책에서 얻은 정보를 바탕으로 '낭만주의 음악의 특징'에 대한 이해를 나타내는 부분으로, 이 역시 '책에서 답을 찾는 질문'에 해당한다. 따라서 이 선택지는 부정확하다.
2번: ⓒ와 ⓓ에는 모두 ‘책에 명시된 내용’에서 질문의 답을 찾아내는 모습이 나타난다. - 이 선택지는 부정확하다. ⓒ는 책의 내용을 종합하여 베토벤의 교향곡 구성의 새로움에 대한 이해를 나타내는 부분으로, 이는 '책에서 답을 찾는 질문'에 해당한